In [12]:
# pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 435.7 kB/s eta 0:00:01
   --------------- ---------------------- 102.4/250.9 kB 845.5 kB/s eta 0:00:01
   ----------------------------------- ---- 225.3/250.9 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\José Roberto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import openpyxl

In [3]:
sumula = pd.read_excel(
    r"C:\Users\José Roberto\Documents\Baygon Geral\Codigo\associacao-futebol\Teste_1_Jogador_Completo\Sumula_Ze.xlsx",
    sheet_name="Sumula",
    header=1,
    engine="calamine"
)
time = pd.read_excel(
    r"C:\Users\José Roberto\Documents\Baygon Geral\Codigo\associacao-futebol\Teste_1_Jogador_Completo\Times_Ze.xlsx",
    sheet_name="Times",
    header=1,
    engine="calamine"
)

In [4]:
# Não está correto o merge por "Boleiro1", foi somente para facilitar/adiantar
duplas = (sumula[["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"]]
          .merge(time, how="left", left_on=["Baba", "Colete", "Boleiro"], right_on=["Baba", "Colete", "Boleiro1"])
          .melt(id_vars=["Baba", "Colete", "Boleiro", "Pizza", "Vitorias", "Derrotas", "Empates"],
    value_vars=["Boleiro1", "Boleiro2", "Boleiro3", "Boleiro4", "Boleiro5", "Boleiro6"],
    var_name="Origem",
    value_name="SegundoBoleiro")
    .rename(columns={"SegundoBoleiro":"Boleiro2"})
    .dropna(subset=["Boleiro2"])
    .drop(columns="Origem"))

duplas = duplas[duplas["Boleiro"] != duplas["Boleiro2"]].reset_index(drop=True)

In [5]:
duplas.head(5)

,Baba,Colete,Boleiro,Pizza,Vitorias,Derrotas,Empates,Boleiro2
0,T10B01,Vermelho,Zé,0,5,3,1,Nelson
1,T10B02,Preto,Zé,0,5,4,0,Abraão
2,T10B03,Cinza,Zé,0,0,6,1,Nelson
3,T10B04,Amarelo,Zé,0,3,0,3,Pig
4,T10B05,Verde,Zé,0,4,5,0,Leléo


In [6]:
grupo_duplas = duplas.groupby(["Boleiro", "Boleiro2"]).agg(
    Total_Pizza=("Pizza", "sum"),
    Total_Vitorias=("Vitorias", "sum"),
    Total_Derrotas=("Derrotas", "sum"),
    Total_Empates=("Empates", "sum"),
    Total_Babas=("Baba", "count")
).reset_index()
grupo_duplas["Total_Partidas"] = grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"] + grupo_duplas["Total_Empates"]
grupo_duplas["P_Vitorias"] = grupo_duplas["Total_Vitorias"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Derrotas"] = grupo_duplas["Total_Derrotas"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_Empates"] = grupo_duplas["Total_Empates"] / grupo_duplas["Total_Partidas"]
grupo_duplas["P_VD"] = grupo_duplas["Total_Vitorias"] / (grupo_duplas["Total_Vitorias"] + grupo_duplas["Total_Derrotas"])
grupo_duplas["P_Pizza"] = grupo_duplas["Total_Pizza"] / grupo_duplas["Total_Babas"]
grupo_duplas = grupo_duplas[
            [
                "Boleiro",
                "Boleiro2",
                "Total_Pizza",
                "Total_Vitorias",
                "Total_Derrotas",
                "Total_Empates",
                "Total_Partidas",
                "Total_Babas",
                "P_Vitorias",
                "P_Derrotas",
                "P_Empates",
                "P_VD",
                "P_Pizza"
            ]
        ]



In [14]:
# grupo_duplas = grupo_duplas.sort_values("P_Vitorias", ascending=False).reset_index(drop=True)
# grupo_duplas = grupo_duplas.sort_values("P_Derrotas", ascending=False).reset_index(drop=True)
# grupo_duplas = grupo_duplas.sort_values("P_VD", ascending=False).reset_index(drop=True)
grupo_duplas = grupo_duplas.sort_values(["Total_Babas", "Total_Partidas", "Total_Vitorias"], ascending=False).reset_index(drop=True)
# grupo_duplas = grupo_duplas.sort_values(["P_Pizza", "P_Vitorias"], ascending=False).reset_index(drop=True)
grupo_duplas.index = grupo_duplas.index + 1
grupo_duplas.to_excel("grupo_duplas.xlsx", index=False)